In [6]:
import csv
import os
import json
import numpy as np
models = [
    # "deepseek-ai/DeepSeek-V3",
    # "deepseek-ai/DeepSeek-R1",
    #"meta-llama/Llama-3.3-70B-Instruct",
    # "meta-llama/Llama-3.1-70B-Instruct",
    #"deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
    # "google/gemma-2-27b-it",
    # "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
    #"Qwen/QwQ-32B-AWQ",
    # "YanAdjeNole/sdoh-llama-3.3-70b"
    # "Qwen/Qwen2.5-14B-Instruct-1M",
    #"meta-llama/Llama-3.1-8B-Instruct",
    # "mistralai/Mistral-7B-Instruct-v0.3",
    #"meta-llama/Llama-3.2-3B-Instruct",
    # "meta-llama/Llama-3.2-1B-Instruct",
    "Qwen/Qwen2.5-1.5B-Instruct"
]

# models = [
#     # "deepseek-ai/DeepSeek-V3",
#     # "deepseek-ai/DeepSeek-R1",
#     #"meta-llama/Llama-3.3-70B-Instruct",
#     # "meta-llama/Llama-3.1-70B-Instruct",
#     #"deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
#     # "google/gemma-2-27b-it",
#     # "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B",
#     #"Qwen/QwQ-32B-AWQ",
#     # "YanAdjeNole/sdoh-llama-3.3-70b"
#     # "Qwen/Qwen2.5-14B-Instruct-1M",
#     "meta-llama/Llama-3.1-8B-Instruct",
#     # "mistralai/Mistral-7B-Instruct-v0.3",
#     #"meta-llama/Llama-3.2-3B-Instruct",
#     # "meta-llama/Llama-3.2-1B-Instruct",
#     "Qwen/Qwen2.5-1.5B-Instruct"
# ]

shot = 0
num_temps = 1
base_path = f"./results/PV_qlora_qwen1p5b_merged_epoch3"

terms = ["code", "sub-code", "span"]
metrics = ["P", "R", "f1"]

def get_list(model_name, temps):
    print(f" num of trials  is {len(temps)}")
    ret = []
    if len(temps) == 1:
        for t in terms:
            line = [model_name, t]
            p_list = []
            r_list = []
            f_list = []
            #for i in range(len(temps)):
            temp = temps[0]
            for m in metrics:
                line.append(round(temp[t][m], 4))
            p_list.append(temp[t]["P"])
            r_list.append(temp[t]["R"])
            f_list.append(temp[t]["f1"])
            # pfinal = f"{round(np.mean(p_list), 4)} +- {round(np.std(p_list), 4)}"
            # rfinal = f"{round(np.mean(r_list), 4)} +- {round(np.std(r_list), 4)}"
            # ffinal = f"{round(np.mean(f_list), 4)} +- {round(np.std(f_list), 4)}"
            # line.append(pfinal)
            # line.append(rfinal)
            # line.append(ffinal)
            ret.append(line)
    else:   
        for t in terms:
            line = [model_name, t]
            p_list = []
            r_list = []
            f_list = []
            for i in range(len(temps)):
                temp = temps[i]
                for m in metrics:
                    #line.append(str(round(100* temp[t][m], 2)))
                    line.append(round(temp[t][m], 4))
                p_list.append(temp[t]["P"])
                r_list.append(temp[t]["R"])
                f_list.append(temp[t]["f1"])
            # pfinal = f"{100* np.mean(p_list)} +- {100* np.std(p_list)}"
            # rfinal = f"{100* np.mean(r_list)} +- {100* np.std(r_list)}"
            # ffinal = f"{100* np.mean(f_list)} +- {100* np.std(f_list)}"
            pfinal = f"{round(np.mean(p_list), 4)} +- {round(np.std(p_list), 4)}"
            rfinal = f"{round(np.mean(r_list), 4)} +- {round(np.std(r_list), 4)}"
            ffinal = f"{round(np.mean(f_list), 4)} +- {round(np.std(f_list), 4)}"
            line.append(pfinal)
            line.append(rfinal)
            line.append(ffinal)
            ret.append(line)
    return ret
    
cols = ["model", "class type"]
if num_temps == 1:
    for m in metrics:
        cols.append(f"{m}")
else:
    for i in range(num_temps + 1):
        #for t in terms:
        for m in metrics:
            cols.append(f"{i}_{m}")
result_list = []
result_list.append(cols)

for model in models:
    #model_name = model.replace("/", "__")
    model_name = model.split("/")[1]
    print (model_name)
    for name in os.listdir(base_path):
        print (name)
        if model_name.lower() in name.lower():
            # select the model
            print (name)
            new_path = os.path.join(base_path, name)
            temps = []
            for res_json in os.listdir(new_path):
                if ".json" in res_json and ".jsonl" not in res_json:
                    with open(os.path.join(base_path, name, res_json), "r") as f:
                        data = json.load(f)
                    if data["n-shot"]["PvExtraction_full"] == shot:
                        temp = data["results"]["PvExtraction_full"]["evaluate_eppc,none"]
                        temps.append(temp)
            assert len(temps) == num_temps
            temp_results = get_list(model_name, temps)    # 3x(2 + 15 + 3)  
            for temp_result in temp_results:
                result_list.append(temp_result)
            
#print (result_list)
with open (f"PV_shot{shot}_results_0102.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(result_list)

Qwen2.5-1.5B-Instruct
__home__lm2445__project_pi_sjf37__lm2445__PV_multiagent__sft__runs__pv_sft_Qwen2.5-1.5B-Instruct_merged_epoch3
__home__lm2445__project_pi_sjf37__lm2445__PV_multiagent__sft__runs__pv_sft_Qwen2.5-1.5B-Instruct_merged_epoch3
 num of trials  is 1
